In [1]:
import pickle

from dotenv import load_dotenv
import numpy as np
import os
load_dotenv()

True

In [2]:
import re
import httpx
from openai import OpenAI
client = OpenAI()
class ChatBot:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})
    
    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result
    
    def execute(self):
        completion = client.chat.completions.create(model="gpt-4o-mini", messages=self.messages)
        return completion.choices[0].message.content


In [3]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:
calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point
syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

simon_blog_search:
e.g. simon_blog_search: Django
Search Simon's blog for that term

Example session:
Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:
Observation: France is a country. The capital is Paris.

You then output:
Answer: The capital of France is Paris
""".strip()

In [4]:
action_re = re.compile('^Action: (\w+): (.*)')

def wikipedia(q):
    response = httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": q,
        "format": "json"
    })
    return response.json()["query"]["search"][0]["snippet"]


def simon_blog_search(q):
    response = httpx.get("https://datasette.simonwillison.net/simonwillisonblog.json", params={
        "sql": """
        select
          blog_entry.title || ': ' || substr(html_strip_tags(blog_entry.body), 0, 1000) as text,
          blog_entry.created
        from
          blog_entry join blog_entry_fts on blog_entry.rowid = blog_entry_fts.rowid
        where
          blog_entry_fts match escape_fts(:q)
        order by
          blog_entry_fts.rank
        limit
          1
        """.strip(),
        "_shape": "array",
        "q": q,
    })
    return response.json()[0]["text"]

def calculate(what):
    print("calculation started")
    return eval(what)


known_actions = {
    "wikipedia": wikipedia,
    "calculate": calculate,
    "simon_blog_search": simon_blog_search
}

In [5]:
bot = ChatBot(prompt)

In [6]:
print(bot.messages)

[{'role': 'system', 'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer.\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point\nsyntax if necessary\n\nwikipedia:\ne.g. wikipedia: Django\nReturns a summary from searching Wikipedia\n\nsimon_blog_search:\ne.g. simon_blog_search: Django\nSearch Simon's blog for that term\n\nExample session:\nQuestion: What is the capital of France?\nThought: I should look up France on Wikipedia\nAction: wikipedia: France\nPAUSE\n\nYou will be called again with this:\nObservation: France is a country. The capital is Paris.\n\nYou then output:\nAnswer: The capital of France is Paris"

In [7]:
next_prompt = "What does England share borders with?"
result = bot(next_prompt)

In [8]:
print(result)

Thought: I should look up information about the geographical borders of England to find out what countries or bodies of water it shares borders with. 
Action: wikipedia: England borders
PAUSE


In [9]:
bot.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer.\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point\nsyntax if necessary\n\nwikipedia:\ne.g. wikipedia: Django\nReturns a summary from searching Wikipedia\n\nsimon_blog_search:\ne.g. simon_blog_search: Django\nSearch Simon's blog for that term\n\nExample session:\nQuestion: What is the capital of France?\nThought: I should look up France on Wikipedia\nAction: wikipedia: France\nPAUSE\n\nYou will be called again with this:\nObservation: France is a country. The capital is Paris.\n\nYou then output:\nAnswer: The capital of France is Pari

In [10]:
actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]

In [11]:
result.split('\n')

['Thought: I should look up information about the geographical borders of England to find out what countries or bodies of water it shares borders with. ',
 'Action: wikipedia: England borders',
 'PAUSE']

In [12]:
actions[0].groups()

('wikipedia', 'England borders')

In [13]:
if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = f"Observation: {observation}"

 -- running wikipedia England borders
Observation: Anglo-Scottish <span class="searchmatch">border</span> (Scottish Gaelic: Crìochan Anglo-Albannach) is an internal <span class="searchmatch">border</span> of the United Kingdom separating Scotland and <span class="searchmatch">England</span> which runs for


In [14]:
result = bot(next_prompt)

In [15]:
print(result)

Answer: England shares a border with Scotland.


In [16]:
bot.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer.\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point\nsyntax if necessary\n\nwikipedia:\ne.g. wikipedia: Django\nReturns a summary from searching Wikipedia\n\nsimon_blog_search:\ne.g. simon_blog_search: Django\nSearch Simon's blog for that term\n\nExample session:\nQuestion: What is the capital of France?\nThought: I should look up France on Wikipedia\nAction: wikipedia: France\nPAUSE\n\nYou will be called again with this:\nObservation: France is a country. The capital is Paris.\n\nYou then output:\nAnswer: The capital of France is Pari

In [17]:
actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = f"Observation: {observation}"
else:
    print(result)

Answer: England shares a border with Scotland.


In [18]:
def query(question, max_turns=5):
    i = 0
    bot = ChatBot(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = f"Observation: {observation}"
        else:
            return result

In [19]:
print(query("what is five multipled by 37"))

Thought: I need to perform a multiplication calculation to find the answer to five multiplied by 37. 
Action: calculate: 5 * 37
PAUSE
 -- running calculate 5 * 37
calculation started
Observation: 185
Answer: Five multiplied by 37 is 185.
Answer: Five multiplied by 37 is 185.


In [20]:
print(query("What does England share borders with?"))

Thought: I should look up England on Wikipedia to find out about its borders and any countries or bodies of water it shares them with.
Action: wikipedia: England borders
PAUSE
 -- running wikipedia England borders
Observation: Anglo-Scottish <span class="searchmatch">border</span> (Scottish Gaelic: Crìochan Anglo-Albannach) is an internal <span class="searchmatch">border</span> of the United Kingdom separating Scotland and <span class="searchmatch">England</span> which runs for
Answer: England shares a border with Scotland.
Answer: England shares a border with Scotland.


In [21]:
print(query("Fifteen * twenty five"))

Thought: I need to calculate the product of fifteen and twenty-five. In numerical terms, that translates to 15 * 25. 
Action: calculate: 15 * 25
PAUSE
 -- running calculate 15 * 25
calculation started
Observation: 375
Answer: Fifteen multiplied by twenty-five is 375.
Answer: Fifteen multiplied by twenty-five is 375.


In [22]:
print(query("Has Simon worked on LLms"))

Thought: I need to check Simon's blog to see if he has mentioned anything about working on LLMs (Large Language Models). 
Action: simon_blog_search: LLMs
PAUSE
 -- running simon_blog_search LLMs
Observation: Language models on the command-line: I gave a talk about accessing Large Language Models from the command-line last week as part of the Mastering LLMs: A Conference For Developers &amp; Data Scientists six week long online conference. The talk focused on my LLM Python command-line utility and ways you can use it (and its plugins) to explore LLMs and use them for useful tasks.

The talk was recorded and is available on YouTube. Here I've turned it into an annotated presentation, with detailed notes and screenshots (there were no slides) to accompany the video.

 

  LLM is a tool I started building last year to help run LLM prompts directly from a command-line terminal. Instructions for installing it are here - you can use pipx install llm or pip install llm or brew install llm.


 